In [ ]:
# Importations nécessaires pour Airflow
from datetime import timedelta, datetime
from airflow.models import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

# Définition des paramètres par défaut pour les tâches de la DAG
default_args = {
    'owner': 'eco_educ',  # Propriétaire de la DAG
    'start_date': datetime(2023, 8, 3),  # Date de début de la DAG
    'retries': 1,  # Nombre de tentatives en cas d'échec
    'retry_delay': timedelta(minutes=1)  # Délai entre les tentatives
}

# Création de la DAG
test_with_dags = DAG('test_with_dags', 
                     default_args=default_args, 
                     description='ETL jobs for Data', 
                     schedule_interval='@daily',  # Fréquence d'exécution quotidienne
                     catchup=False,  # Ignorer les exécutions manquées
                     tags=['example', 'testdags']  # Tags pour la DAG
                    )

# Fonction pour extraire les données
def extract_jobs():
    # Lien du fichier à extraire
    apps = extract_datas('')  # Appel à une fonction "extract_datas"
    return apps

# Appel de la fonction d'extraction
gapps, greviews = extract_jobs()

# Fonction pour effectuer les transformations sur les données
def transform_jobs(df1):
    # Fonction pour effectuer les transformations nécessaires
    # prenant en paramètre le DataFrame df1
    return extract_to_csv(df1)  # Appel à une fonction "extract_to_csv"

# Fonction pour charger les données transformées
def load_jobs():
    # Lien pour l'upload des données
    return uploadToS3('subject', '')  # Appel à une fonction "uploadToS3"

# Création des opérateurs de tâches
start_task = DummyOperator(task_id='start_task', dag=test_with_dags)
extract_dags = PythonOperator(task_id='extract_dags', python_callable=extract_jobs, dag=test_with_dags)
transform_dags = PythonOperator(task_id='transform_dags', python_callable=transform_jobs, op_args=gapps, dag=test_with_dags)
load_dags = PythonOperator(task_id='load_dags', python_callable=load_jobs, dag=test_with_dags)
end_task = DummyOperator(task_id='end_task', dag=test_with_dags)

# Définition des dépendances entre les tâches
start_task >> extract_dags >> transform_dags >> load_dags >> end_task
